In [2]:
# 程序功能：利用OpenCV实现人脸跟踪，并用卡尔曼滤波器跟踪人脸位置。
# 输入：视频文件input_video.mp4
# 输出：视频文件output_video.mp4
# 运行环境：Python 3, OpenCV

import cv2
import numpy as np

# 初始化人脸检测器
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 初始化卡尔曼滤波器参数
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                      [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                     [0, 1, 0, 1],
                                     [0, 0, 1, 0],
                                     [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32) * 0.03

# 读取视频
cap = cv2.VideoCapture('input_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 转换为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 人脸检测
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # 用矩形标注人脸位置
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # 更新卡尔曼滤波器
        measurement = np.array([[x + w / 2], [y + h / 2]], np.float32)
        prediction = kalman.predict()
        kalman.correct(measurement)

        # 获取预测的位置
        pt = (int(prediction[0]), int(prediction[1]))
        
        # 在预测位置处打码
        cv2.rectangle(frame, (pt[0] - w // 2, pt[1] - h // 2), (pt[0] + w // 2, pt[1] + h // 2), (0, 0, 0), -1)

    cv2.imshow('Face Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
